In [2]:
import xarray as xr
from xarray import DataArray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import OSM
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd

# Temperature index models

## Acknowledgment

by Thomas Gölles 2024

## Naive first approach

$$
\sum_{i=1}^{n} M_i = DDF \sum_{i=1}^{n} T^{+} \Delta t
$$

In [3]:
degree_day_factor = 8.0  # mm/day/degree celsius
temperature_plus = 3.0  # degree celsius
delta_time = 4  # day

melt = degree_day_factor * temperature_plus * delta_time
print(f"melt of snow for {delta_time} days is {melt} mm")

melt of snow for 4 days is 96.0 mm


Now lets look at the cumulative melt over 10 days

In [4]:
n = 11
days = range(1, n)
melt = 0

for day in days:
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(f"cummulative melt of snow for {day} is {melt:.2f} mm")

cummulative melt of snow for 1 is 96.00 mm
cummulative melt of snow for 2 is 192.00 mm
cummulative melt of snow for 3 is 288.00 mm
cummulative melt of snow for 4 is 384.00 mm
cummulative melt of snow for 5 is 480.00 mm
cummulative melt of snow for 6 is 576.00 mm
cummulative melt of snow for 7 is 672.00 mm
cummulative melt of snow for 8 is 768.00 mm
cummulative melt of snow for 9 is 864.00 mm
cummulative melt of snow for 10 is 960.00 mm


ok a bit nicer, but the Temperature is still constant. Now use a random tempearature list form -5 to +10

In [8]:
n = 11
days = range(1, n)

np.random.seed(42)
temperatures = np.random.uniform(-5, 10, len(days))
total_melt = 0

for day in days:
    temperature_plus = temperatures[day - 1]
    melt = degree_day_factor * temperature_plus * delta_time
    total_melt = total_melt + degree_day_factor * temperature_plus * delta_time
    print(
        f"melt of snow for {day} with {temperature_plus:.2f}°C is {melt:.2f} mm  and {total_melt:.2f} mm in total "
    )

melt of snow for 1 with 0.62°C is 19.78 mm  and 19.78 mm in total 
melt of snow for 2 with 9.26°C is 296.34 mm  and 316.12 mm in total 
melt of snow for 3 with 5.98°C is 191.36 mm  and 507.48 mm in total 
melt of snow for 4 with 3.98°C is 127.36 mm  and 634.84 mm in total 
melt of snow for 5 with -2.66°C is -85.11 mm  and 549.72 mm in total 
melt of snow for 6 with -2.66°C is -85.12 mm  and 464.60 mm in total 
melt of snow for 7 with -4.13°C is -132.12 mm  and 332.48 mm in total 
melt of snow for 8 with 7.99°C is 255.76 mm  and 588.25 mm in total 
melt of snow for 9 with 4.02°C is 128.54 mm  and 716.78 mm in total 
melt of snow for 10 with 5.62°C is 179.87 mm  and 896.66 mm in total 


Can you spot the error? Look at those days wiht negative temperatures.

In [11]:
n = 11
days = range(1, n)

np.random.seed(42)
temperatures = np.random.uniform(-5, 10, len(days))
total_melt = 0

for day in days:
    current_temperature = temperatures[day - 1]
    temperature_plus = np.maximum(current_temperature, 0)
    melt = degree_day_factor * temperature_plus * delta_time
    total_melt = total_melt + degree_day_factor * temperature_plus * delta_time
    print(
        f"melt of snow for day {day} with {temperature_plus:.2f}°C is {melt:.2f} mm and {total_melt:.2f} mm in total "
    )

melt of snow for day 1 with 0.62°C is 19.78 mm and 19.78 mm in total 
melt of snow for day 2 with 9.26°C is 296.34 mm and 316.12 mm in total 
melt of snow for day 3 with 5.98°C is 191.36 mm and 507.48 mm in total 
melt of snow for day 4 with 3.98°C is 127.36 mm and 634.84 mm in total 
melt of snow for day 5 with 0.00°C is 0.00 mm and 634.84 mm in total 
melt of snow for day 6 with 0.00°C is 0.00 mm and 634.84 mm in total 
melt of snow for day 7 with 0.00°C is 0.00 mm and 634.84 mm in total 
melt of snow for day 8 with 7.99°C is 255.76 mm and 890.60 mm in total 
melt of snow for day 9 with 4.02°C is 128.54 mm and 1019.14 mm in total 
melt of snow for day 10 with 5.62°C is 179.87 mm and 1199.01 mm in total 


## Better use of Python

If you often reuse a culation but it in a function.
A function starts with "def" then the function name you want to have followed by the input argumetnts in ().

Here I also used so called typehints like degree_day_factor: float. Which means that we expect a floating point number as the degree day factor. In Python this optional but helps a lot in understanding what is expected.

Here I also give a default value of the threshold_temp. This means that if no third argument is given the standard value of 0.0 is used.

In [ ]:
def degree_day_model(
    temperatures: np.ndarray | DataArray,
    degree_day_factor: float,
    threshold_temp: float = 0.0,
) -> np.ndarray | DataArray:
    """
    Estimate daily melt amount using the Degree-Day Model.

    Parameters:
    temperatures (numpy.array): Array of daily mean air temperatures (°C).
    degree_day_factor (float): Degree-day factor (mm °C^-1 day^-1).
    threshold_temp (float): Threshold temperature above which melting occurs (°C). Default is 0.0°C.

    Returns:
    numpy.array: Array of daily melt amounts (mm).
    """
    # Calculate positive temperature differences (temperatures above the threshold)
    positive_temperatures = np.maximum(temperatures - threshold_temp, 0)

    # Calculate daily melt amounts
    daily_melt = positive_temperatures * degree_day_factor

    return daily_melt

In [ ]:
temperatures = np.array([-3.2, -1.1, 0.0, 2.2, 5.1, 3.0, 0.0, -2.0])

In [ ]:
ddf = 8.0
melt_amounts = degree_day_model(temperatures, degree_day_factor=ddf)
melt_amounts

note here we used the default value for threshold_temp.
So the result is an array of values correspondig to daily temperatures.
To make it nicer to work with lets make a pandas dataframe together with the temperatures

In [ ]:
from operator import index


melt = pd.DataFrame(
    melt_amounts,
    index=temperatures,
    columns=[f"melt (mm)"],
)
melt.index.name = "Temperature (°C)"
melt

In [ ]:
melt.plot()

### Exersize

* plot the curve for different degree day factors.
* compare snow and ice: 2.5 to 11.6 mm/(day K)  for snow 6.6 to 20.0 mm/(day K) for ice,
* change the function to use SI base units

What is the total amount of melt in mm over all days

In [ ]:
melt["melt (mm)"].sum()

## Spatial distributed version

We take actual weather data from geosphere over the Pasterze region.

![INCA data auschnitt](figures/Pasterze.png)

Load the netcdf dataset

In [ ]:
inca_pasterze = xr.open_dataset("data/inca_pasterze.nc")

In [ ]:
inca_pasterze

take onhly the Tempearature at 2 meter varaible called T2M, for one hour to get the spatial extend. 
Note the dimensions. Ther are 192 time sclices and 9 points in y and 8 points in x direction.
The INCA dataset has a resolutution of 1x1 km. Which also determines the resolution on which we can calculate the melt.

IMPORTANT: Since our model expects Day as the time unit we need to be carefull and take daily averages. Always think about the units of input and outputs!

In [ ]:
# Compute daily averages
daily_averages = inca_pasterze.resample(time="1D").mean()

In [ ]:
temperatures = daily_averages["T2M"]
temperature_example = temperatures.sel(time="2023-07-01T00:00:00")
temperature_example.plot()

In [ ]:
type(temperature_example)

In [ ]:
melt_amounts_distributed = degree_day_model(temperature_example, degree_day_factor=ddf)
melt_amounts_distributed.name = "melt per day (mm)"

In [ ]:
melt_amounts_distributed.plot()

## Now with space and time

We have data over 8 days lets see the resulting melt

In [ ]:
tempeartures = daily_averages["T2M"]
tempeartures

The original

In [ ]:
melt_result = degree_day_model(temperatures, degree_day_factor)
melt_result.name = "melt (mm)"

In [ ]:
melt_result

Thats the result. Now we can plot and animate the result.

Basically this an open street map as a background and then makes a plot for each timestep. The result is then saved as gif.
It takes about a minute to run.

**WARNING Do not run this below on binder, it takes up to much memory.**

In [ ]:
# Initialize the figure
fig = plt.figure(figsize=(10, 10))
osm_tiles = OSM()
ax = plt.axes(projection=osm_tiles.crs)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=":")

# Pre-calculate extent to avoid recalculating it each frame
lat_min, lat_max = np.min(melt_result.lat.values), np.max(melt_result.lat.values)
lon_min, lon_max = np.min(melt_result.lon.values), np.max(melt_result.lon.values)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add base map once (assumes all slices have similar extents)
ax.add_image(osm_tiles, 12)

# Prepare colorbar (setup with dummy data)
sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(vmin=0, vmax=150))
sm._A = []  # dummy data for ScalarMappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, aspect=20, label="Melt per day (mm)")


# The update function for the animation
previous_collections = []  # Store previous plot references


def update(frame):
    global previous_collections  # Refer to the global list
    # Remove previous collections
    for collection in previous_collections:
        collection.remove()
    previous_collections.clear()  # Clear the list after removal

    data_slice = melt_result.isel(time=frame)
    new_collection = ax.pcolormesh(
        data_slice.lon,
        data_slice.lat,
        data_slice.values,
        transform=ccrs.PlateCarree(),
        cmap="viridis",
        alpha=0.5,
    )
    previous_collections.append(new_collection)  # Add new plot to the list
    time_formatted = pd.to_datetime(data_slice.time.values).strftime("%Y-%m-%d")
    ax.set_title(f"Day: {time_formatted}")


# Create animation
ani = FuncAnimation(fig, update, frames=len(melt_result.time), interval=200)

ani.save("figures/melt_rate_animation.gif", dpi=300)